In [23]:
import numpy as np
import pandas as pd

In [24]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn import neighbors, model_selection, tree
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

In [29]:
dataset = pd.read_csv('all_stocks_5yr.csv', delimiter=',')
#se eliminan las filas que tienen valores nulos de cara a realizar posteriores estudios
#HABRIA QUE TENERLO IMPUTADO CON VALORES PERDIDOS!!
dataset.dropna(axis=0, inplace = True) 

In [30]:
atSalida = 'close'
atEntrada = ['open','high','low','volume']

dataset_output = dataset[atSalida].copy()
dataset.drop(atSalida, axis=1, inplace=True)
dataset = dataset[atEntrada].copy()

Por tanto , las columnas nulas son open, high y  low

# 3.3 Transformación de variables categóricas a numéricas

No se va a realizar dicho tipo de transformación , pues para el proyecto se realizará principalmente redes neuronales y regresión polinomial.

# 3.4 Reducción de datos

AGREGAR DESPUES VARIABLE FECHA!!!!!

# Obtención del rendimiento con todas las variables

Para comprobar la calidad de las técnicas de selección de variables, se va a calcular el rendimiento de una regresión polinomial  si utilizamos todas las variables. Nos hemos decidido por esta técnica, pues es fácil de implementar.

Se aplicará el método hold-out para obtener los conjuntos de entrenamiento y de test (70% de datos para entrenar).



SE ELIMINARAN LAS FILAS ASOCIADAS A LOS VALORES NULOS PARA PODER REALIZAR LAS COSAS!!! JUNTAR CON LO DE IVAN!

Rendimiento con regresión polinomial sobre todas las variables

In [31]:
X_train, X_test, y_train, y_test = train_test_split(dataset, dataset_output, test_size=0.2, random_state=12)

In [32]:
def regresion_polinomial(dataset,dataset_output, p_degree):
    # Generamos los conjuntos de entrenamiento y de test
    X_train, X_test, y_train, y_test = train_test_split(dataset, dataset_output, test_size=0.2, random_state=12)
    # Creamos una Pipeline en la que generamos variables polinómicas de grado 2, estandarizamos los datos y aprendemos una regresión lineal
    pipePolinomial = Pipeline([('polynomial', PolynomialFeatures(degree=p_degree)),('scaler', StandardScaler()),('regressor', LinearRegression())])
    # Entrenamos la Pipeline
    pipePolinomial.fit(X_train, y_train)
    # Obtenemos el rendimiento en entrenamiento y en test (MSE)
    prTrain = pipePolinomial.predict(X_train)
    errorTrain = mean_squared_error(y_train, prTrain)
    prTest = pipePolinomial.predict(X_test)
    errorTest = mean_squared_error(y_test, prTest)
    print('El error en entrenamiento es {:.4f} y en test es {:.4f}'.format(errorTrain, errorTest))
regresion_polinomial(dataset,dataset_output,2)

El error en entrenamiento es 0.4663 y en test es 0.4903


## Análisis de correlaciones

In [ ]:
import matplotlib.pyplot as plt
correlaciones = X_train.corr(method ='pearson')
correlaciones

In [ ]:
# Mostramos la matriz de correlaciones especificando el rango de los valores [-1, 1]
fig = plt.figure(figsize=(50,50))
ax = fig.add_subplot(111)
cax = ax.matshow(correlaciones, vmin=-1, vmax=1, cmap=plt.cm.rainbow)
fig.colorbar(cax)
ticks = np.arange(0,len(X_train.columns),1)
ax.set_xticks(ticks)
ax.set_yticks(ticks)
# Añadimos los nombres de las variables en la figura
names = X_train.columns
ax.set_xticklabels(names, rotation='90')
ax.set_yticklabels(names)
plt.show()

Tras la obtención de matriz de correlaciones, se puede determinar que las variables de entrada están relacionadas entre si de manera muy alta, salvo el volumen.

Veamos ahora el rendimiento que se obtendría al eliminar una de éstas variables. Para hacer más completo el experimento, se procederá a crear 3 datasets distintos: cada uno eliminando una variable distinta.

*Eliminación* *de* *open*, *high*, *low* por separado

In [ ]:
dataset1 = dataset.copy()
dataset_output1 = dataset_output.copy()

In [ ]:
variables = ['open', 'high', 'low']   
for nombrevar in variables:
    X_train, X_test, y_train, y_test = train_test_split(dataset1, dataset_output1, test_size=0.2, random_state=12)
    X_train = X_train.drop([nombrevar],axis=1)
    X_test = X_test.drop([nombrevar],axis=1)

    pipePolinomial = Pipeline([('polynomial', PolynomialFeatures(degree=2)),('scaler', StandardScaler()),('regressor', LinearRegression())])
    pipePolinomial.fit(X_train, y_train)

    #Se obtiene el rendimiento en entrenamiento y en test (MSE)
    prTrain = pipePolinomial.predict(X_train)
    errorTrain = mean_squared_error(y_train, prTrain)
    prTest = pipePolinomial.predict(X_test)
    errorTest = mean_squared_error(y_test, prTest)
    print(f"Resutados para la eliminacion de {nombrevar}")
    print('El error en entrenamiento es {:.4f} y en test es {:.4f}'.format(errorTrain, errorTest))
    print()

*Eliminación* *por* *pares*

In [ ]:
pares = [ ['open', 'high'] , ['open', 'low'] , ['high', 'low']]   
for par in pares:
    X_train, X_test, y_train, y_test = train_test_split(dataset1, dataset_output1, test_size=0.2, random_state=12)
    
    X_train = X_train.drop([par[0]],axis=1)
    X_train = X_train.drop([par[1]],axis=1)

    X_test = X_test.drop([par[0]],axis=1)
    X_test = X_test.drop([par[1]],axis=1)


    pipePolinomial = Pipeline([('polynomial', PolynomialFeatures(degree=2)),('scaler', StandardScaler()),('regressor', LinearRegression())])
    pipePolinomial.fit(X_train, y_train)

    #Se obtiene el rendimiento en entrenamiento y en test (MSE)
    prTrain = pipePolinomial.predict(X_train)
    errorTrain = mean_squared_error(y_train, prTrain)
    prTest = pipePolinomial.predict(X_test)
    errorTest = mean_squared_error(y_test, prTest)
    print(f"Resutados para la eliminacion de {par[0]} y {par[1]}")
    print('El error en entrenamiento es {:.4f} y en test es {:.4f}'.format(errorTrain, errorTest))
    print()

*Eliminación* *de* *las* *tres* *variables*

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dataset1, dataset_output1, test_size=0.2, random_state=12)
    
X_train = X_train.drop(['open'] ,axis=1)
X_train = X_train.drop(['low'],axis=1)
X_train = X_train.drop(['high'] ,axis=1)

X_test = X_test.drop(['open'] ,axis=1)
X_test = X_test.drop(['low'],axis=1)
X_test = X_test.drop(['high'] ,axis=1)

pipePolinomial = Pipeline([('polynomial', PolynomialFeatures(degree=2)),('scaler', StandardScaler()),('regressor', LinearRegression())])
pipePolinomial.fit(X_train, y_train)

#Se obtiene el rendimiento en entrenamiento y en test (MSE)
prTrain = pipePolinomial.predict(X_train)
errorTrain = mean_squared_error(y_train, prTrain)
prTest = pipePolinomial.predict(X_test)
errorTest = mean_squared_error(y_test, prTest)
print("Resultado para la eliminacion de high, low, y open")
print('El error en entrenamiento es {:.4f} y en test es {:.4f}'.format(errorTrain, errorTest))
print()

Tras los experimentos realizados, se puede observar que se obtiene un peor resultado al tratar de eliminar cualquiera de las tres variables que cuentan con una mayor correlación -casi perfecta- .

Por otro lado, en caso de eliminar una variable,la que menos impacto tiene en el rendimiento del modelo es 'open', y la que más influye es 'high'. 

En caso de eliminar dos variables,el mayor impacto negativo en el rendimiento vendrá dado por la eliminación de conjunta de 'low' y 'high'. Cosa que es natural, pues representa el valor más bajo y más alto de las acciones de una empresa en un día.

La eliminación de 'high', 'low' y 'open' provoca un desenlace nefasto y esperado, pues predecir el valor al final del día de las acciones a partir del volumen de acciones negociadas es sumamente complicado 

# Comparativas de métodos

In [25]:
from sklearn.metrics import mean_squared_error

# LSTM

 La característica principal de las redes recurrentes es que la información puede persistir introduciendo bucles en el diagrama de la red, por lo que, básicamente, pueden «recordar» estados previos y utilizar esta información para decidir cuál será el siguiente.

De manera sencilla, pueden ser pensadas como múltiples copias de la misma red, cada una pasando un mensaje a su sucesor. Por lo que, gracias a su naturaleza, pueden ser relacionadas con listas o secuencias.

Las LSTM pueden aprender dependencias largas, por lo que se podría decir que tienen una «memoria» a más largo plazo. Lo que les hace atractivas para este tipo de problemas, donde la información pasada puede jugar un papel importante.

DIBUJO!!!!

In [33]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [34]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [35]:
#importing the packages 
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM

ImportError: Keras requires TensorFlow 2.2 or higher. Install TensorFlow via `pip install tensorflow`

In [36]:
#Lectura de datos
todataframe = pd.read_csv('all_stocks_5yr.csv', delimiter=',')
todataframe.dropna(axis=0, inplace = True) 

La comparativa del método se hará con la empresa AAl

In [37]:
df = todataframe[todataframe['Name']=='AAL']

In [38]:
#Crear un conjunto de entrenamiento y de test (70% para train, 30% para test)
numero_ejemplos_entrenamiento = int(len(df)*0.7)
training_set = df.iloc[:numero_ejemplos_entrenamiento, 1:2].values
test_set = df.iloc[numero_ejemplos_entrenamiento:, 1:2].values

In [39]:
# Feature Scaling
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)# Creating a data structure with 60 time-steps and 1 output
X_train = []
y_train = []
for i in range(60, 800):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))


In [40]:
model = Sequential()#Adding the first LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
model.add(Dropout(0.2))# Adding a second LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))# Adding a third LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))# Adding a fourth LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50))
model.add(Dropout(0.2))# Adding the output layer
model.add(Dense(units = 1))

# Compiling the RNN
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
model.fit(X_train, y_train, epochs = 100, batch_size = 32)

NameError: name 'Sequential' is not defined

In [ ]:
# Getting the predicted stock price of 2017
dataset_train = df.iloc[:numero_ejemplos_entrenamiento, 1:2]
dataset_test = df.iloc[numero_ejemplos_entrenamiento:, 1:2]dataset_total = pd.concat((dataset_train, dataset_test), axis = 0)inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].valuesinputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []
for i in range(60, 519):
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [ ]:
predicted_stock_price = model.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

-------------------------------------------

In [56]:
#Creacion del dataframe
seriesdata = todataframe.sort_index(ascending=True, axis=0)
new_seriesdata = pd.DataFrame(index=range(0,len(todataframe)),columns=['date','close'])
length_of_data=len(seriesdata)
for i in range(0,length_of_data):
    new_seriesdata['date'][i] = seriesdata['date'][i]
    new_seriesdata['close'][i] = seriesdata['close'][i]

    
#Colocar los indices de nuevo
new_seriesdata.index = new_seriesdata.date
new_seriesdata.drop('date', axis=1, inplace=True)

In [60]:
#Crear un conjunto de entrenamiento y de test (70% para train, 30% para test)
numero_ejemplos_entrenamiento = int(len(new_seriesdata)*0.7)

#Crear conjuntos de train y test con el total de los elementos
myseriesdataset = new_seriesdata.values
totrain = myseriesdataset[0:numero_ejemplos_entrenamiento,:] #conjunto de ejemplos para entrenar
tovalid = myseriesdataset[numero_ejemplos_entrenamiento:,:] #conjunto de ejemplos para validar

In [62]:
#Convertir el dataset en x_train e y_train
scalerdata = MinMaxScaler(feature_range=(0, 1))
scale_data = scalerdata.fit_transform(myseriesdataset)
x_totrain, y_totrain = [], []
length_of_totrain=len(totrain)
for i in range(60,length_of_totrain):
    x_totrain.append(scale_data[i-60:i,0])
    y_totrain.append(scale_data[i,0])
x_totrain, y_totrain = np.array(x_totrain), np.array(y_totrain)
x_totrain = np.reshape(x_totrain, (x_totrain.shape[0],x_totrain.shape[1],1))

#LSTM neural network configuracion
lstm_model = Sequential()
lstm_model.add(LSTM(units=50, return_sequences=True, input_shape=(x_totrain.shape[1],1)))
lstm_model.add(LSTM(units=50))
lstm_model.add(Dense(1))
lstm_model.compile(loss='mean_squared_error', optimizer='adadelta', metrics= ['mean_squared_error'])
lstm_model.fit(x_totrain, y_totrain, epochs=3, batch_size=1, verbose=2)

Epoch 1/3
821/821 - 25s - loss: 0.2391 - mean_squared_error: 0.2391
Epoch 2/3
821/821 - 22s - loss: 0.1745 - mean_squared_error: 0.1745
Epoch 3/3
821/821 - 23s - loss: 0.1148 - mean_squared_error: 0.1148


In [69]:
#Predecir
myinputs = new_seriesdata[len(new_seriesdata) - (len(tovalid)) - 60:].values

In [70]:
myinputs  = scalerdata.transform(myinputs)

In [71]:
tostore_test_result = []
for i in range(60,myinputs.shape[0]):
    tostore_test_result.append(myinputs[i-60:i,0])
tostore_test_result = np.array(tostore_test_result)
tostore_test_result = np.reshape(tostore_test_result,(tostore_test_result.shape[0],tostore_test_result.shape[1],1))
myclosing_priceresult = lstm_model.predict(tostore_test_result)
myclosing_priceresult


array([[0.2491232 ],
       [0.24908283],
       [0.24899095],
       [0.24907307],
       [0.24926008],
       [0.24975638],
       [0.25067288],
       [0.25178325],
       [0.2529819 ],
       [0.2541405 ],
       [0.25509575],
       [0.255988  ],
       [0.25688314],
       [0.25776976],
       [0.2584892 ],
       [0.25905514],
       [0.25974008],
       [0.26029006],
       [0.26070642],
       [0.26110286],
       [0.26159295],
       [0.2626034 ],
       [0.26408416],
       [0.26557723],
       [0.26696166],
       [0.26796892],
       [0.26843005],
       [0.26852897],
       [0.2681647 ],
       [0.2675032 ],
       [0.26661348],
       [0.26548728],
       [0.2644061 ],
       [0.2634856 ],
       [0.2625374 ],
       [0.26188037],
       [0.26140264],
       [0.26104096],
       [0.26097092],
       [0.26138306],
       [0.26207218],
       [0.26296067],
       [0.26401982],
       [0.26503006],
       [0.2662667 ],
       [0.26743725],
       [0.26829156],
       [0.269

In [72]:
prediccion_resultante = scalerdata.inverse_transform(myclosing_priceresult)

In [73]:
prediccion_resultante.shape

(378, 1)

In [74]:
tovalid.shape

(378, 1)

Resultado del rendimiento

In [78]:
print(f"El MSE obtenido ha sido de : {mean_squared_error(tovalid, prediccion_resultante)}")

El MSE obtenido ha sido de : 345.25766857289824


# Regresión polinomial

In [41]:
todataframe = pd.read_csv('all_stocks_5yr.csv', delimiter=',')
todataframe.dropna(axis=0, inplace = True) 
todataframe = todataframe[todataframe['Name']=='AAL']

In [43]:
todataframe.head()

,date,open,high,low,close,volume,Name
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL


In [44]:
atSalida = 'close'
atEntrada = ['open','high','low','volume']

todataframe_output = todataframe[atSalida].copy()
todataframe.drop(atSalida, axis=1, inplace=True)
todataframe = todataframe[atEntrada].copy()

In [46]:
grados= [2,4,6]

for grado in grados:
    print(f"Ejecucion apra regresion polinomial de grado {grado}")
    regresion_polinomial(todataframe,todataframe_output,grado)
    print("------------------------------------------------------")

Ejecucion apra regresion polinomial de grado 2
El error en entrenamiento es 0.1138 y en test es 0.1101
------------------------------------------------------
Ejecucion apra regresion polinomial de grado 4
El error en entrenamiento es 0.0945 y en test es 0.9647
------------------------------------------------------
Ejecucion apra regresion polinomial de grado 6
El error en entrenamiento es 0.0663 y en test es 864.2716
------------------------------------------------------


*Observaciones*:

**Conclusiones** **sobre** **comparativa** **de** **métodos**

Similar rendimiento con conjuntos de entrenamiento con regresion polinomial y LSTM 